# 데이터



( train, test , bus_bts 공통 사항)

해당 데이터에는 버스카드를 통해 결제를 한 경우에 대한 정류소 승, 하차 데이터로 모든 승차정보의 경우는 기록이 되어있지만, 

버스에서 하차를 할 때, 버스카드를 찍지 않는 경우, 해당 기록이 비어 있는 상태입니다. 따라서, 승차 인원수와 하차 인원수가 동일하지 않고 다소 차이가 있음을 미리 알려드립니다.



(train, test csv 공통사항)

해당 버스정류장에 대한 각각의 위도, 경도가 제공이 되어있는 상태로 같은 정류장 이름이지만 위도와 경도가 서로 다른 경우가 존재합니다. 

해당 경우는, 같은 정류장 이름을 가지고 있는 길 건너편의 정류장에 해당이 됩니다.

 



 1. train.csv and test.csv

 

train.csv 의 경우, 2019년 9월 제주도의 각 날짜, 출근시간(6시~12시)의 버스 정류장별 승하차 인원, 퇴근시간(18시~20시)의 버스 정류장별 승차 인원이 기록되어 있습니다.

test.csv의 경우, 2019년 10월의 각 날짜, 출근시간(오전 6시~12시)의 버스 정류장별 승하차 인원이 기록되어 있습니다.



2. bus_bts.csv

 

bus_bts의 경우, 버스카드별로 승하차 정보가 기록이 되어있습니다. 해당 데이터는 탑승 시간대가오전 6시부터 12시 사이인 경우만 있습니다.



3.     submission_제출양식.csv 

 

submission_제출양식의 경우, test data의 ID와 목표변수인 18시~20시 승차 인원로만 이루어져 있습니다.

참가자분들께서는 test.csv에서 ID와 예측값을 결합하여, 해당 submission_제출양식.csv 파일형식 처럼 만든 이후,

해당 제출 파일 양식 그대로(변수명 포함된 상태) 제출을 해주시면 됩니다



추가 사용 가능 데이터

      a) (8회 kcb 시각화 대회의 제주도 금융라이프 데이터(jeju_financial_life_data.csv)

      b) 행정동별 위도 경도 데이터 https://dacon.io/?module=file&act=procFileDownload&file_srl=244977&sid=8cd192144ab18be0e02e4a16f851729b&module_srl=228539  (H - 행정동, B - 법정동)


train.csv & test.csv
1. id: 해당 데이터에서의 고유한 ID(train, test와의 중복은 없음)
2. date: 날짜
3. bus_route_id: 노선ID 
4. in_out: 시내버스, 시외버스 구분
5. station_code: 해당 승하차 정류소의 ID
6. station_name: 해당 승하차 정류소의 이름
7. latitude: 해당 버스 정류장의 위도 (같은 정류장 이름이어도 버스의 진행 방향에 따라 다를 수 있음)
8. longitude: 해당 버스 정류장의 경도 (같은 정류장 이름이어도 버스의 진행 방향에 따라 다를 수 있음)
9. X~Y_ride: X:00:00부터 X:59:59까지 승차한 인원 수
10. X_Y_takeoff: X:00:00부터 X:59:59까지 하차한 인원 수
11. 18~20_ride: 18:00:00부터 19:59:59까지 승차한 인원 수 (train data에만 존재)


# Data_Path, import

In [1]:
!pip show missingno

Name: missingno
Version: 0.4.2
Summary: Missing data visualization module for Python.
Home-page: https://github.com/ResidentMario/missingno
Author: Aleksey Bilogur
Author-email: aleksey.bilogur@gmail.com
License: UNKNOWN
Location: c:\users\playdata\anaconda3\lib\site-packages
Requires: matplotlib, numpy, scipy, seaborn
Required-by: 


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
sns.set(font_scale=2.5)

import missingno as msno
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [3]:
data_path = 'C:/Users/Playdata/Downloads/data13/'

# bus_bts, submission_sample, test, train, coordinate -> coordinate_jeju(제주좌표)

In [4]:
bus_bts = pd.read_csv(data_path + 'bus_bts.csv')
bus_bts

,user_card_id,bus_route_id,vhc_id,geton_date,geton_time,geton_station_code,geton_station_name,getoff_date,getoff_time,getoff_station_code,getoff_station_name,user_category,user_count
0,1.010010e+15,23000000,149793674,2019-09-10,06:34:45,360,노형오거리,2019-09-10,07:10:31,592.0,화북초등학교,1,1
1,1.010010e+15,23000000,149793674,2019-09-10,06:34:58,360,노형오거리,2019-09-10,06:56:27,3273.0,고산동산(광양방면),1,1
2,1.019160e+15,21420000,149793535,2019-09-10,07:19:07,2495,동광환승정류장4(제주방면),2019-09-10,07:40:29,431.0,정존마을,4,1
3,1.019150e+15,21420000,149793512,2019-09-09,09:14:47,3282,대정환승정류장(대정읍사무소),2019-09-09,10:02:46,431.0,정존마을,1,1
4,1.010010e+15,21420000,149793512,2019-09-09,09:28:53,2820,삼정지에듀,2019-09-09,10:21:37,2972.0,제주국제공항(종점),4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2409409,6.573162e+15,30420000,149797565,2019-10-16,07:08:31,1937,L마트,NaN,NaN,NaN,NaN,1,1
2409410,9.441160e+15,30420000,149797565,2019-10-16,07:16:31,1908,한라산교회,NaN,NaN,NaN,NaN,1,1
2409411,9.446038e+15,25070000,149797565,2019-10-16,08:29:05,1882,고도농원,NaN,NaN,NaN,NaN,1,1
2409412,4.309690e+15,25070000,149797565,2019-10-16,08:40:32,1938,주공아파트5단지,NaN,NaN,NaN,NaN,1,1


In [5]:
submission_sample = pd.read_csv(data_path + 'submission_sample.csv')
submission_sample

,id,18~20_ride
0,415423,0
1,415424,0
2,415425,0
3,415426,0
4,415427,0
...,...,...
228165,643588,0
228166,643589,0
228167,643590,0
228168,643591,0


In [6]:
test = pd.read_csv(data_path + 'test.csv')
test

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff
0,415423,2019-10-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,4.0,4.0,7.0,2.0,9.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,415424,2019-10-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,6.0,6.0,1.0,8.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0
2,415425,2019-10-01,4270000,시외,432,정존마을,33.48181,126.47352,2.0,4.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,415426,2019-10-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.49252,1.0,11.0,18.0,8.0,26.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0
4,415427,2019-10-01,4270000,시내,1636,롯데호텔,33.24872,126.41032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228165,643588,2019-10-16,32820000,시내,786,고산환승정류장(고산1리),33.30073,126.18044,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
228166,643589,2019-10-16,32820000,시내,1080,애월고등학교,33.46262,126.33447,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
228167,643590,2019-10-16,32820000,시내,1129,한림환승정류장(한림리),33.41437,126.26336,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
228168,643591,2019-10-16,32820000,시내,1564,제주시외버스터미널,33.49946,126.51479,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
train = pd.read_csv(data_path + 'train.csv')
train

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,...,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride
0,0,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,0.0,1.0,...,5.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2019-09-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,4.0,...,2.0,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
2,2,2019-09-01,4270000,시외,432,정존마을,33.48181,126.47352,1.0,1.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,3,2019-09-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.49252,0.0,17.0,...,26.0,14.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0
4,4,2019-09-01,4270000,시내,1646,중문관광단지입구,33.25579,126.41260,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415418,415418,2019-09-30,32820000,시내,1129,한림환승정류장(한림리),33.41437,126.26336,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
415419,415419,2019-09-30,32820000,시내,1564,제주시외버스터미널,33.49946,126.51479,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
415420,415420,2019-09-30,32820000,시내,2322,해병부대,33.23100,126.26273,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
415421,415421,2019-09-30,32820000,시내,3291,애월환승정류장(애월리),33.46483,126.31870,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
coordinate = pd.read_excel(data_path + '행정_법정동 중심좌표.xlsx')
coordinate

,코드,시도,시군구,읍면동,하위,위도,경도,코드종류
0,1111051500,서울특별시,종로구,청운효자동,NaN,37.584009,126.970626,H
1,1111053000,서울특별시,종로구,사직동,NaN,37.575408,126.965944,H
2,1111054000,서울특별시,종로구,삼청동,NaN,37.590758,126.980996,H
3,1111055000,서울특별시,종로구,부암동,NaN,37.594768,126.965574,H
4,1111056000,서울특별시,종로구,평창동,NaN,37.613029,126.974485,H
...,...,...,...,...,...,...,...,...
22214,5013032022,제주특별자치도,서귀포시,표선면,하천리,33.348068,126.832421,B
22215,5013032023,제주특별자치도,서귀포시,표선면,성읍리,33.393144,126.794822,B
22216,5013032024,제주특별자치도,서귀포시,표선면,가시리,33.354784,126.771072,B
22217,5013032025,제주특별자치도,서귀포시,표선면,세화리,33.321147,126.798867,B


In [12]:
coordinate_jeju = coordinate[coordinate['시도'] == '제주특별자치도']
coordinate_jeju

,코드,시도,시군구,읍면동,하위,위도,경도,코드종류
3503,5011025000,제주특별자치도,제주시,한림읍,NaN,33.376347,126.284734,H
3504,5011025300,제주특별자치도,제주시,애월읍,NaN,33.411040,126.393958,H
3505,5011025600,제주특별자치도,제주시,구좌읍,NaN,33.498076,126.795431,H
3506,5011025900,제주특별자치도,제주시,조천읍,NaN,33.472170,126.667445,H
3507,5011031000,제주특별자치도,제주시,한경면,NaN,33.322497,126.223161,H
...,...,...,...,...,...,...,...,...
22214,5013032022,제주특별자치도,서귀포시,표선면,하천리,33.348068,126.832421,B
22215,5013032023,제주특별자치도,서귀포시,표선면,성읍리,33.393144,126.794822,B
22216,5013032024,제주특별자치도,서귀포시,표선면,가시리,33.354784,126.771072,B
22217,5013032025,제주특별자치도,서귀포시,표선면,세화리,33.321147,126.798867,B


# data_list

In [13]:
data_list = [bus_bts, test, train,]

In [14]:
for i in data_list:
  print(i.describe())
  print(i.head())
  print(i.tail())
  print(i.info())

       user_card_id  bus_route_id        vhc_id  geton_station_code  \
count  2.409414e+06  2.409414e+06  2.409414e+06        2.409414e+06   
mean   2.920959e+15  2.545786e+07  1.497948e+08        1.197044e+05   
std    2.249502e+16  4.017593e+06  1.808600e+03        8.422198e+05   
min    1.355555e+07  4.270000e+06  1.497911e+08        0.000000e+00   
25%    1.014140e+15  2.313000e+07  1.497932e+08        2.780000e+02   
50%    1.019150e+15  2.400000e+07  1.497936e+08        6.030000e+02   
75%    5.188320e+15  2.870000e+07  1.497970e+08        2.057000e+03   
max    1.000700e+19  3.282000e+07  1.497989e+08        6.115101e+06   

       getoff_station_code  user_category    user_count  
count         1.513678e+06   2.409414e+06  2.409414e+06  
mean          8.406106e+04   4.073226e+00  1.012212e+00  
std           7.058707e+05   6.523765e+00  1.354882e-01  
min           1.000000e+00   1.000000e+00  1.000000e+00  
25%           2.860000e+02   1.000000e+00  1.000000e+00  
50%         

                  id  bus_route_id  station_code       latitude  \
count  415423.000000  4.154230e+05  4.154230e+05  415423.000000   
mean   207711.000000  2.691935e+07  4.209311e+04      33.420375   
std    119922.434776  3.924652e+06  4.971504e+05       0.107996   
min         0.000000  4.270000e+06  1.000000e+00      33.208350   
25%    103855.500000  2.346000e+07  3.200000e+02      33.291080   
50%    207711.000000  2.803000e+07  1.130000e+03      33.478850   
75%    311566.500000  3.003000e+07  2.226000e+03      33.500020   
max    415422.000000  3.282000e+07  6.115101e+06      33.963640   

           longitude       6~7_ride       7~8_ride       8~9_ride  \
count  415423.000000  415423.000000  415423.000000  415423.000000   
mean      126.533574       0.305893       0.829699       0.815350   
std         0.140986       1.109766       2.255116       2.317561   
min       126.165040       0.000000       0.000000       0.000000   
25%       126.475780       0.000000       0.000000 

# Missing Value Preprocessing

In [15]:
for i in data_list:  
  print(i.isnull().sum())

user_card_id                0
bus_route_id                0
vhc_id                      0
geton_date                  0
geton_time                  0
geton_station_code          0
geton_station_name         49
getoff_date            895736
getoff_time            895736
getoff_station_code    895736
getoff_station_name    895775
user_category               0
user_count                  0
dtype: int64
id               0
date             0
bus_route_id     0
in_out           0
station_code     0
station_name     0
latitude         0
longitude        0
6~7_ride         0
7~8_ride         0
8~9_ride         0
9~10_ride        0
10~11_ride       0
11~12_ride       0
6~7_takeoff      0
7~8_takeoff      0
8~9_takeoff      0
9~10_takeoff     0
10~11_takeoff    0
11~12_takeoff    0
dtype: int64
id               0
date             0
bus_route_id     0
in_out           0
station_code     0
station_name     0
latitude         0
longitude        0
6~7_ride         0
7~8_ride         0
8~9_ride      

In [16]:
coordinate_jeju_df = coordinate['하위'].fillna('리')
coordinate_jeju_df
coordinate_jeju_df.isnull().sum()

0

In [17]:
bus_bts.isnull().sum()

user_card_id                0
bus_route_id                0
vhc_id                      0
geton_date                  0
geton_time                  0
geton_station_code          0
geton_station_name         49
getoff_date            895736
getoff_time            895736
getoff_station_code    895736
getoff_station_name    895775
user_category               0
user_count                  0
dtype: int64

In [18]:
bus_bts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2409414 entries, 0 to 2409413
Data columns (total 13 columns):
 #   Column               Dtype  
---  ------               -----  
 0   user_card_id         float64
 1   bus_route_id         int64  
 2   vhc_id               int64  
 3   geton_date           object 
 4   geton_time           object 
 5   geton_station_code   int64  
 6   geton_station_name   object 
 7   getoff_date          object 
 8   getoff_time          object 
 9   getoff_station_code  float64
 10  getoff_station_name  object 
 11  user_category        int64  
 12  user_count           int64  
dtypes: float64(2), int64(5), object(6)
memory usage: 239.0+ MB


In [19]:
train.head()

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,...,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride
0,0,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,0.0,1.0,...,5.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2019-09-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,4.0,...,2.0,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
2,2,2019-09-01,4270000,시외,432,정존마을,33.48181,126.47352,1.0,1.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,3,2019-09-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.49252,0.0,17.0,...,26.0,14.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0
4,4,2019-09-01,4270000,시내,1646,중문관광단지입구,33.25579,126.41260,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


## 전처리데이터 => 저장명 예시 : train -> train_df


In [20]:
train['date'].agg(['min','max']) #2019-09-01 ~ 2019-09-30 (train)

min    2019-09-01
max    2019-09-30
Name: date, dtype: object

In [21]:
test['date'].agg(['min','max']) #2019-10-01 ~ 2019-10-16 (test)

min    2019-10-01
max    2019-10-16
Name: date, dtype: object

In [22]:
train['date'].unique()

array(['2019-09-01', '2019-09-02', '2019-09-03', '2019-09-04',
       '2019-09-05', '2019-09-06', '2019-09-07', '2019-09-08',
       '2019-09-09', '2019-09-10', '2019-09-11', '2019-09-12',
       '2019-09-13', '2019-09-14', '2019-09-15', '2019-09-16',
       '2019-09-17', '2019-09-18', '2019-09-19', '2019-09-20',
       '2019-09-21', '2019-09-22', '2019-09-23', '2019-09-24',
       '2019-09-25', '2019-09-26', '2019-09-27', '2019-09-28',
       '2019-09-29', '2019-09-30'], dtype=object)

In [23]:
train['date'] = pd.to_datetime(train['date'])
train['weekday'] = train['date'].dt.weekday

test['date'] = pd.to_datetime(test['date'])
test['weekday'] = test['date'].dt.weekday

train.shape, test.shape

((415423, 22), (228170, 21))

In [24]:
train.head()

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,...,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride,weekday
0,0,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,0.0,1.0,...,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6
1,1,2019-09-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,4.0,...,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,6
2,2,2019-09-01,4270000,시외,432,정존마을,33.48181,126.47352,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,6
3,3,2019-09-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.49252,0.0,17.0,...,14.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0,6
4,4,2019-09-01,4270000,시내,1646,중문관광단지입구,33.25579,126.41260,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6


In [25]:
train.tail()

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,...,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride,weekday
415418,415418,2019-09-30,32820000,시내,1129,한림환승정류장(한림리),33.41437,126.26336,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
415419,415419,2019-09-30,32820000,시내,1564,제주시외버스터미널,33.49946,126.51479,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
415420,415420,2019-09-30,32820000,시내,2322,해병부대,33.23100,126.26273,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
415421,415421,2019-09-30,32820000,시내,3291,애월환승정류장(애월리),33.46483,126.31870,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
415422,415422,2019-09-30,32820000,시내,6115100,서귀포시외버스터미널,33.24873,126.50799,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0


In [26]:
train['weekday']

0         6
1         6
2         6
3         6
4         6
         ..
415418    0
415419    0
415420    0
415421    0
415422    0
Name: weekday, Length: 415423, dtype: int64

In [27]:
test['weekday']

0         1
1         1
2         1
3         1
4         1
         ..
228165    2
228166    2
228167    2
228168    2
228169    2
Name: weekday, Length: 228170, dtype: int64